In [1]:
import cv2 as cv
import numpy as np
import os
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from joblib import dump, load
from sklearn.neural_network import MLPClassifier
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from skimage.filters import prewitt_h,prewitt_v
import matplotlib.pyplot as plt
from joblib import dump, load
from PIL import Image, ImageFilter, ImageChops

In [2]:
def read_images(input_path):
    """
    Read images in the input_path, 
    save image, patient of each image and the class (group/labels)
    
    Params: 
    input_path = path to the original images 
    
    Return:
    images = list of all images
    labels = list with class for each image
    """
   
    # Lists to save images, patients and labels
    images = []
    labels = []

    # Browse input path
    for class_dir in os.listdir(input_path):
        class_path = os.path.join(input_path, class_dir)

        # If it is a directory 
        if os.path.isdir(class_path):      

            for image_file in os.listdir(class_path):
                image_path = os.path.join(class_path, image_file)

                image = cv.imread(image_path)

                # Append image, patient id and class to list
                images.append(image)            
                labels.append(class_dir)    
                
    df = pd.DataFrame({'image': images, 'label': labels})
    
    # Create a LabelEncoder object
    le = LabelEncoder()
    le.fit(df['label'])
    df['label'] = le.transform(df['label'])
    
    df = df.reset_index()
                
    return (images, labels)

In [3]:
x_train, y_train = read_images("/home/vsa/ufpr/cb/ufpr-bioinspired-comp/dataset/train")
x_test, y_test = read_images("/home/vsa/ufpr/cb/ufpr-bioinspired-comp/dataset/test")

In [4]:
def extract_grayscale(data):
    
    ft_gray = []
    
    for image in data:
        
        shape = image.shape
        
        gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        ft_gray.append(np.reshape(gray, (shape[0]*shape[1])))
    
    return ft_gray

In [5]:
def extract_mean_channels(data):
    
    ft_mean = []

    for image in data:

        shape = image.shape
        matrix = np.zeros(shape) 
        #print(shape, matrix.shape, end=' ')
        matrix = np.mean(image, axis=2)
        
        ft_mean.append(np.reshape(matrix, (shape[0]*shape[1]))) 
        #print(np.reshape(matrix, (shape[0], shape[1])).shape)
            
    return ft_mean     

In [123]:
from skimage import feature

def extract_edges_len(data):
    
    ft_edges_len = []
    
    for image in data:
        
#         shape = image.shape
#         img = image.reshape((-1,3))
#         img = np.float32(img)

#         criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
#         K = 5
#         attempts=10

#         ret, label, center = cv.kmeans(tw, K, None, criteria, attempts,cv.KMEANS_PP_CENTERS)
#         center = np.uint8(center)
#         res = center[label.flatten()]
#         segmented = res.reshape((shape))      
     
#         scale = 1
#         delta = 0
#         ddepth = cv.CV_16S

#         grad_x = cv.Sobel(segmented, ddepth, 1, 0, ksize=3, scale=scale, delta=delta, borderType=cv.BORDER_DEFAULT)
#         grad_y = cv.Sobel(segmented, ddepth, 0, 1, ksize=3, scale=scale, delta=delta, borderType=cv.BORDER_DEFAULT)

#         abs_grad_x = cv.convertScaleAbs(grad_x)
#         abs_grad_y = cv.convertScaleAbs(grad_y)

#         grad = cv.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)

#         pil = Image.fromarray(grad)
        
#         edges = pil.filter(ImageFilter.FIND_EDGES)
#         edges = np.asarray(edges)
        
        feature.canny(cv.cvtColor(image, cv.COLOR_BGR2GRAY))
        # Extract the edge pixels
        edge_pixels = edges.nonzero()

        # Calculate the edge length for each edge pixel
        edge_lengths = []
        
        for i, j in zip(*edge_pixels):
            # Get the coordinates of the current pixel
            x, y = j, i

            # Check if there's a next pixel in the edge segment
            if x > 0:
                # Get the coordinates of the next pixel
                x2 = edge_pixels[0][x - 1]
                y2 = edge_pixels[1][x - 1]
            else:
                # Handle the edge case where there's no next pixel
                x2 = edge_pixels[0][-1]
                y2 = edge_pixels[1][-1]

            # Calculate the distance between the two pixels
            distance = np.sqrt((x2 - x)**2 + (y2 - y)**2)

            # Add the distance to the list of edge lengths
            edge_lengths.append(distance)
            
        ft_edges_len.append(edge_lengths) 
      
    return ft_edges_len

In [6]:
ft_gray = extract_grayscale(x_train)

In [7]:
ft_mean = extract_mean_channels(x_train)

In [ ]:
ft_edges_len = extract_edges_len(x_train)

In [10]:
len(ft_gray[0])

262144

In [11]:
dump(ft_gray, 'ft_gray.joblib')
dump(ft_mean, 'ft_mean.joblib')
dump(ft_edges_len, 'ft_edges_len.joblib')

['ft_mean.joblib']

In [ ]:
ft_gray = load('ft_gray.joblib')
ft_mean = load('ft_mean.joblib')

In [ ]:
# Grayscale Pixel Values done
# Mean Pixel Value of Channels
# length of the edges